In [36]:
import urllib.request
import requests
from urllib.parse import urlparse,unquote
from bs4 import BeautifulSoup
from multiprocessing.dummy import Pool as ThreadPool
import os


'''
find all zip files from a page and download them all
target webpage should be FTP like, meaning, all downloadable links are enclosed in href tag

For example:
https://www2.census.gov/geo/tiger/TIGER2021/BG/
https://downloads-direct.freemdict.com/Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/A1%E5%90%AC%E5%8A%9B/
'''

archive_url="https://downloads-direct.freemdict.com/Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/"
# link = urllib.request.urlopen(archive_url)
# r = link.read()
# soup = BeautifulSoup(r, 'html.parser')

# Create two lists, one for links one for filenames
# Used to create link-name paires used in Pool
# Pool is used so we can download multifiles at the sametime

target_files = []
filenames = []
file_extension=".mp4"

to_visit = [archive_url]  # Queue of URLs to visit, starting with the root

while to_visit:
    current_url = to_visit.pop(0)  # Get the next URL to visit
    try:
        response = requests.get(current_url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            for link in soup.find_all('a', href=True):
                href = link['href']
                # Avoid going up in the directory structure
                if href.endswith('/') and href != '../':
                    to_visit.append(current_url+href)  # Properly join URLs
                elif href.endswith(file_extension):
                    full_path = current_url + href
                    target_files.append(full_path)
                    # Create a relative file path based on the URL structure
                    parsed_url = urlparse(current_url)
                    file_path = parsed_url.path + href if href != '../' else ''
                    filenames.append(file_path.lstrip('/'))
                    print(filenames)
    except Exception as e:
        print(f"Failed to process {current_url}: {e}")

downloads_dir = ".\\downloads\\"
if not os.path.exists(downloads_dir):
    os.makedirs(downloads_dir)

def get_file(link, file_path):
    # Decode URL-encoded characters in the file_path
    decoded_path = unquote(file_path)
    # Determine the local file path, preserving the directory structure
    local_path = os.path.join(downloads_dir, decoded_path)
    # Ensure the subdirectory exists
    os.makedirs(os.path.dirname(local_path), exist_ok=True)
    # Download the file to the specified location
    urllib.request.urlretrieve(link, local_path)

pool = ThreadPool(5)
pool.starmap(get_file, zip(target_files,filenames))

['Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/1/129_1_%E7%96%91%E9%97%AE%E5%BD%A2%E5%AE%B9%E8%AF%8Dquel_%E7%96%91%E9%97%AE%E8%AF%8D.mp4']
['Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/1/129_1_%E7%96%91%E9%97%AE%E5%BD%A2%E5%AE%B9%E8%AF%8Dquel_%E7%96%91%E9%97%AE%E8%AF%8D.mp4', 'Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/1/130_1_%E5%AE%9E%E7%94%A8%E5%8D%95%E8%AF%8D.mp4']
['Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/1/129_1_%E7%96%91%E9%97%AE%E5%BD%A2%E5%AE%B9%E8%AF%8Dquel_%E7%96%91%E9%97%AE%E8%AF%8D.mp4', 'Language_

['Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/1/129_1_%E7%96%91%E9%97%AE%E5%BD%A2%E5%AE%B9%E8%AF%8Dquel_%E7%96%91%E9%97%AE%E8%AF%8D.mp4', 'Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/1/130_1_%E5%AE%9E%E7%94%A8%E5%8D%95%E8%AF%8D.mp4', 'Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/1/131_1_%E8%AE%B2%E8%BF%B0%E5%88%9A%E5%85%A5%E5%AD%A6%E7%9A%84%E4%BA%A4%E9%99%85%E7%94%A8%E8%AF%AD.mp4', 'Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/10/163_1_%E9%87%8D%E7%82%B9%E8%AF%AD%E6%B3%95%EF%BC%88%E4%B8%80%EF%BC%89_%E5%BD%A2%E5%AE%B9

['Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/1/129_1_%E7%96%91%E9%97%AE%E5%BD%A2%E5%AE%B9%E8%AF%8Dquel_%E7%96%91%E9%97%AE%E8%AF%8D.mp4', 'Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/1/130_1_%E5%AE%9E%E7%94%A8%E5%8D%95%E8%AF%8D.mp4', 'Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/1/131_1_%E8%AE%B2%E8%BF%B0%E5%88%9A%E5%85%A5%E5%AD%A6%E7%9A%84%E4%BA%A4%E9%99%85%E7%94%A8%E8%AF%AD.mp4', 'Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/10/163_1_%E9%87%8D%E7%82%B9%E8%AF%AD%E6%B3%95%EF%BC%88%E4%B8%80%EF%BC%89_%E5%BD%A2%E5%AE%B9

['Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/1/129_1_%E7%96%91%E9%97%AE%E5%BD%A2%E5%AE%B9%E8%AF%8Dquel_%E7%96%91%E9%97%AE%E8%AF%8D.mp4', 'Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/1/130_1_%E5%AE%9E%E7%94%A8%E5%8D%95%E8%AF%8D.mp4', 'Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/1/131_1_%E8%AE%B2%E8%BF%B0%E5%88%9A%E5%85%A5%E5%AD%A6%E7%9A%84%E4%BA%A4%E9%99%85%E7%94%A8%E8%AF%AD.mp4', 'Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/10/163_1_%E9%87%8D%E7%82%B9%E8%AF%AD%E6%B3%95%EF%BC%88%E4%B8%80%EF%BC%89_%E5%BD%A2%E5%AE%B9

['Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/1/129_1_%E7%96%91%E9%97%AE%E5%BD%A2%E5%AE%B9%E8%AF%8Dquel_%E7%96%91%E9%97%AE%E8%AF%8D.mp4', 'Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/1/130_1_%E5%AE%9E%E7%94%A8%E5%8D%95%E8%AF%8D.mp4', 'Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/1/131_1_%E8%AE%B2%E8%BF%B0%E5%88%9A%E5%85%A5%E5%AD%A6%E7%9A%84%E4%BA%A4%E9%99%85%E7%94%A8%E8%AF%AD.mp4', 'Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/10/163_1_%E9%87%8D%E7%82%B9%E8%AF%AD%E6%B3%95%EF%BC%88%E4%B8%80%EF%BC%89_%E5%BD%A2%E5%AE%B9

...

['Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/1/129_1_%E7%96%91%E9%97%AE%E5%BD%A2%E5%AE%B9%E8%AF%8Dquel_%E7%96%91%E9%97%AE%E8%AF%8D.mp4', 'Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/1/130_1_%E5%AE%9E%E7%94%A8%E5%8D%95%E8%AF%8D.mp4', 'Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/1/131_1_%E8%AE%B2%E8%BF%B0%E5%88%9A%E5%85%A5%E5%AD%A6%E7%9A%84%E4%BA%A4%E9%99%85%E7%94%A8%E8%AF%AD.mp4', 'Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/10/163_1_%E9%87%8D%E7%82%B9%E8%AF%AD%E6%B3%95%EF%BC%88%E4%B8%80%EF%BC%89_%E5%BD%A2%E5%AE%B9

['Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/1/129_1_%E7%96%91%E9%97%AE%E5%BD%A2%E5%AE%B9%E8%AF%8Dquel_%E7%96%91%E9%97%AE%E8%AF%8D.mp4', 'Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/1/130_1_%E5%AE%9E%E7%94%A8%E5%8D%95%E8%AF%8D.mp4', 'Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/1/131_1_%E8%AE%B2%E8%BF%B0%E5%88%9A%E5%85%A5%E5%AD%A6%E7%9A%84%E4%BA%A4%E9%99%85%E7%94%A8%E8%AF%AD.mp4', 'Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/10/163_1_%E9%87%8D%E7%82%B9%E8%AF%AD%E6%B3%95%EF%BC%88%E4%B8%80%EF%BC%89_%E5%BD%A2%E5%AE%B9

['Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/1/129_1_%E7%96%91%E9%97%AE%E5%BD%A2%E5%AE%B9%E8%AF%8Dquel_%E7%96%91%E9%97%AE%E8%AF%8D.mp4', 'Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/1/130_1_%E5%AE%9E%E7%94%A8%E5%8D%95%E8%AF%8D.mp4', 'Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/1/131_1_%E8%AE%B2%E8%BF%B0%E5%88%9A%E5%85%A5%E5%AD%A6%E7%9A%84%E4%BA%A4%E9%99%85%E7%94%A8%E8%AF%AD.mp4', 'Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/10/163_1_%E9%87%8D%E7%82%B9%E8%AF%AD%E6%B3%95%EF%BC%88%E4%B8%80%EF%BC%89_%E5%BD%A2%E5%AE%B9

['Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/1/129_1_%E7%96%91%E9%97%AE%E5%BD%A2%E5%AE%B9%E8%AF%8Dquel_%E7%96%91%E9%97%AE%E8%AF%8D.mp4', 'Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/1/130_1_%E5%AE%9E%E7%94%A8%E5%8D%95%E8%AF%8D.mp4', 'Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/1/131_1_%E8%AE%B2%E8%BF%B0%E5%88%9A%E5%85%A5%E5%AD%A6%E7%9A%84%E4%BA%A4%E9%99%85%E7%94%A8%E8%AF%AD.mp4', 'Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%ADA2%20%20124%E8%AF%BE%E6%97%B6%E5%85%A8/1---20/10/163_1_%E9%87%8D%E7%82%B9%E8%AF%AD%E6%B3%95%EF%BC%88%E4%B8%80%EF%BC%89_%E5%BD%A2%E5%AE%B9

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]